# PID 控制器

`PID控制器 (PIDController)` 类提供了一个标准的比例-积分-微分控制器的实现。PID控制是一种无处不在的反馈控制机制，广泛应用于工业控制系统和各种需要连续调制控制的应用中。本Jupyter Notebook解释了PID控制器的组成部分，并演示了其在控制一个简单模拟系统中的有效性。

## PID控制的工作原理

PID控制器持续计算一个`误差 (error)`值，该值是期望的`设定点 (setpoint)`与测量的`过程变量 (process_variable)`之间的差值。它试图通过调整一个`控制信号 (control_signal)`来随时间最小化这个误差。

- **比例 (P) 项:** 输出与当前误差成正比。一个高的P增益会导致对误差的强烈响应，但也可能导致不稳定。
- **积分 (I) 项:** 该项累积过去的误差。它旨在消除纯P控制器可能出现的稳态误差。
- **微分 (D) 项:** 该项响应误差的变化率。它具有阻尼效应，可以减少超调并提高稳定性。

### 抗积分饱和 (Anti-Windup)
此实现包含一个称为**抗积分饱和**的重要特性。当控制器的输出饱和（达到其最大或最小限制）时，积分项可能会不受控制地增长，这种现象称为“饱和”。当系统最终脱离限制时，这可能导致显著的超调。我们的控制器通过在输出饱和时停止积分来防止这种情况。

## 仿真示例

在这个例子中，我们将使用`PIDController`来管理一个简单模拟水箱的水位。该水箱有一个入流（由PID控制）和一个恒定的出流（泄漏）。目标是维持水位在一个特定的设定点。我们还将引入一个扰动（泄漏突然增加），以观察控制器如何适应。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` 和 `numpy` (`pip install matplotlib numpy`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.local_agents.control.pid_controller import PIDController
from swp.core.interfaces import State

# 一个简单的水箱仿真（我们的'被控对象'）
class SimpleTank:
    def __init__(self, initial_level, area, leak_rate):
        self.level = initial_level
        self.area = area
        self.leak_rate = leak_rate
        
    def update(self, inflow, dt):
        delta_level = (inflow - self.leak_rate) / self.area * dt
        self.level += delta_level
        return self.level

# --- 仿真设置 ---
setpoint = 10.0 # 目标水位 (m)
pid = PIDController(
    Kp=1.2, Ki=0.1, Kd=0.5,
    setpoint=setpoint,
    min_output=0.0,  # 入流不能为负
    max_output=10.0  # 最大水泵入流速率
)

tank = SimpleTank(initial_level=0.0, area=100.0, leak_rate=2.0)

# --- 仿真循环 ---
dt = 1.0
duration = 200
history = {'time': [], 'level': [], 'control_signal': [], 'setpoint': []}

for t in range(int(duration/dt)):
    # 在中途引入一个扰动
    if t * dt > duration / 2:
        tank.leak_rate = 4.0
        
    # 控制器计算动作
    current_state = State(process_variable=tank.level)
    control_signal = pid.compute_control_action(current_state, dt)
    
    # 被控对象被更新
    tank.update(control_signal, dt)
    
    # 记录历史
    history['time'].append(t * dt)
    history['level'].append(tank.level)
    history['control_signal'].append(control_signal)
    history['setpoint'].append(setpoint)

# --- 绘图 ---
fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(history['time'], history['level'], 'b-', label='水箱水位 (Tank Level)')
ax1.plot(history['time'], history['setpoint'], 'k--', label='设定点 (Setpoint)')
ax1.axvline(x=duration/2, color='r', linestyle='--', label='扰动 (增加泄漏)')
ax1.set_xlabel('时间 (s)')
ax1.set_ylabel('水位 (m)', color='b')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_title('水箱的PID控制')
ax1.legend(loc='lower right')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(history['time'], history['control_signal'], 'g:', label='控制信号 (入流)')
ax2.set_ylabel('入流 (m³/s)', color='g')
ax2.tick_params(axis='y', labelcolor='g')
ax2.legend(loc='upper right')

fig.tight_layout()
plt.show()
